##### Copyright 2020 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Playing CartPole with the Actor-Critic Method


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/reinforcement_learning/actor_critic">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/reinforcement_learning/actor_critic.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/reinforcement_learning/actor_critic.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/reinforcement_learning/actor_critic.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial demonstrates how to implement the [Actor-Critic](https://papers.nips.cc/paper/1786-actor-critic-algorithms.pdf) method using TensorFlow to train an agent on the [Open AI Gym](https://gym.openai.com/) CartPole-V0 environment.
The reader is assumed to have some familiarity with [policy gradient methods](https://papers.nips.cc/paper/1713-policy-gradient-methods-for-reinforcement-learning-with-function-approximation.pdf) of reinforcement learning. 


**Actor-Critic methods**

Actor-Critic methods are [temporal difference (TD) learning](https://en.wikipedia.org/wiki/Temporal_difference_learning) methods that represent the policy function independent of the value function. 

A policy function (or policy) returns a probability distribution over actions that the agent can take based on the given state.
A value function determines the expected return for an agent starting at a given state and acting according to a particular policy forever after.

In the Actor-Critic method, the policy is referred to as the *actor* that proposes a set of possible actions given a state, and the estimated value function is referred to as the *critic*, which evaluates actions taken by the *actor* based on the given policy.

In this tutorial, both the *Actor* and *Critic* will be represented using one neural network with two outputs.


**CartPole-v0**

In the [CartPole-v0 environment](https://gym.openai.com/envs/CartPole-v0), a pole is attached to a cart moving along a frictionless track. 
The pole starts upright and the goal of the agent is to prevent it from falling over by applying a force of -1 or +1 to the cart. 
A reward of +1 is given for every time step the pole remains upright.
An episode ends when (1) the pole is more than 15 degrees from vertical or (2) the cart moves more than 2.4 units from the center.

<center>
  <figure>
    <image src="images/cartpole-v0.gif">
    <figcaption>
      Trained actor-critic model in Cartpole-v0 environment
    </figcaption>
  </figure>
</center>


The problem is considered "solved" when the average total reward for the episode reaches 195 over 100 consecutive trials.

## Setup

Import necessary packages and configure global settings.


In [1]:
import collections
import gym
import numpy as np
import statistics
import tensorflow as tf
import tqdm
import glob
import random

from matplotlib import pyplot as plt
from tensorflow.keras import layers
from typing import Any, List, Sequence, Tuple

import gym
import os


# Create the environment
env = gym.make("LunarLander-v2")
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Set seed for experiment reproducibility
seed = 42
env.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

# Small epsilon value for stabilizing division operations
eps = np.finfo(np.float32).eps.item()

writer = tf.summary.create_file_writer("/home/kimbring2/Supervised_A2C/tensorboard")

2022-02-23 13:39:20.595125: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-02-23 13:39:23.535977: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-23 13:39:23.538683: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-02-23 13:39:23.561863: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-02-23 13:39:23.561895: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: kimbring2-ROG-Strix-GA35DX-G35DX
2022-02-23 13:39:23.561902: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: kimbring2-ROG-Strix-GA35DX-G35DX
2022-02-23 13:39:23.562006: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 

## Model

The *Actor* and *Critic* will be modeled using one neural network that generates the action probabilities and critic value respectively. This tutorial uses model subclassing to define the model. 

During the forward pass, the model will take in the state as the input and will output both action probabilities and critic value $V$, which models the state-dependent [value function](https://spinningup.openai.com/en/latest/spinningup/rl_intro.html#value-functions). The goal is to train a model that chooses actions based on a policy $\pi$ that maximizes expected [return](https://spinningup.openai.com/en/latest/spinningup/rl_intro.html#reward-and-return).

For Cartpole-v0, there are four values representing the state: cart position, cart-velocity, pole angle and pole velocity respectively. The agent can take two actions to push the cart left (0) and right (1) respectively.

Refer to [OpenAI Gym's CartPole-v0 wiki page](http://www.derongliu.org/adp/adp-cdrom/Barto1983.pdf) for more information.


In [2]:
class ActorCritic(tf.keras.Model):
  """Combined actor-critic network."""

  def __init__(
      self, 
      num_actions: int, 
      num_hidden_units: int):
    """Initialize."""
    super().__init__()

    self.common_1 = layers.Dense(num_hidden_units*2, activation="relu")
    self.common_2 = layers.Dense(num_hidden_units, activation="relu")
    self.actor = layers.Dense(num_actions)
    self.critic = layers.Dense(1)

  def call(self, inputs: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
    x = self.common_1(inputs)
    x = self.common_2(x)
    
    return self.actor(x), self.critic(x)

In [3]:
num_actions = env.action_space.n  # 2
num_hidden_units = 256

model = ActorCritic(num_actions, num_hidden_units)
sl_model = ActorCritic(num_actions, num_hidden_units)

sl_model.load_weights("/home/kimbring2/Supervised_A2C/LunaLander_SL_Model/variables/variables")

## Training

To train the agent, you will follow these steps:

1. Run the agent on the environment to collect training data per episode.
2. Compute expected return at each time step.
3. Compute the loss for the combined actor-critic model.
4. Compute gradients and update network parameters.
5. Repeat 1-4 until either success criterion or max episodes has been reached.


### 1. Collecting training data

As in supervised learning, in order to train the actor-critic model, you need
to have training data. However, in order to collect such data, the model would
need to be "run" in the environment.

Training data is collected for each episode. Then at each time step, the model's forward pass will be run on the environment's state in order to generate action probabilities and the critic value based on the current policy parameterized by the model's weights.

The next action will be sampled from the action probabilities generated by the model, which would then be applied to the environment, causing the next state and reward to be generated.

This process is implemented in the `run_episode` function, which uses TensorFlow operations so that it can later be compiled into a TensorFlow graph for faster training. Note that `tf.TensorArray`s were used to support Tensor iteration on variable length arrays.

In [4]:
# Wrap OpenAI Gym's `env.step` call as an operation in a TensorFlow function.
# This would allow it to be included in a callable TensorFlow graph.

def env_step(action: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
  """Returns state, reward and done flag given an action."""
  #env.render()

  state, reward, done, _ = env.step(action)
  return (state.astype(np.float32), 
          np.array(reward, np.int32), 
          np.array(done, np.int32))


def tf_env_step(action: tf.Tensor) -> List[tf.Tensor]:
  return tf.numpy_function(env_step, [action], 
                           [tf.float32, tf.int32, tf.int32])

In [5]:
import glob
import random
import numpy as np

file_list = glob.glob("/home/kimbring2/Supervised_A2C/expert_data/*.npy")
file = random.choice(file_list)
  
data = np.load(file, allow_pickle=True)
data = np.reshape(data, 1)
data = data[0]
   
state_list = data['state']
action_list = data['action'] 
reward_list = data['reward']
done_list = data['done']
next_state_list = data['next_state']

In [6]:
def run_supervised_episode(
    model: tf.keras.Model) -> Tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
  """Runs a single episode to collect training data."""

  action_logits = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
  actions = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)

  file_list = glob.glob("/home/kimbring2/Supervised_A2C/expert_data/*.npy")
  file = random.choice(file_list)
  
  data = np.load(file, allow_pickle=True)
  data = np.reshape(data, 1)
  data = data[0]
    
  state_list = data['state']
  action_list = data['action'] 
  reward_list = data['reward']
  done_list = data['done']
  next_state_list = data['next_state']
  
  reward_sum = 0
  for t in range(0, len(data['state']) - 1):
    # Convert state into a batched tensor (batch size = 1)
    #state = tf.expand_dims(state_list[t], 0)
    state = state_list[t]
  
    # Run the model and to get action probabilities and critic value
    action_logits_t, value = model(state)
    actions = actions.write(t, action_list[t])
    
    reward_sum += reward_list[t]
    
    # Store log probability of the action chosen
    action_logits = action_logits.write(t, action_logits_t[0])
  
  action_logits = action_logits.stack()
  actions = actions.stack()

  return action_logits, actions

In [7]:
import tensorflow_probability as tfp
tfd = tfp.distributions


def run_reinforcement_episode(
    initial_state: tf.Tensor,  
    initial_reward_sum: tf.Tensor,  
    model: tf.keras.Model, 
    max_steps: int) -> Tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
  """Runs a single episode to collect training data."""

  action_logits_ts = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)  
  sl_action_logits_ts = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)  

  action_logits = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
  values = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
  rewards = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)

  initial_state_shape = initial_state.shape
  state = initial_state
    
  reward_sum_shape = initial_reward_sum.shape
  reward_sum = initial_reward_sum
  for t in tf.range(max_steps):
    # Convert state into a batched tensor (batch size = 1)
    state = tf.expand_dims(state, 0)
  
    # Run the model and to get action probabilities and critic value
    action_logits_t, value = model(state)
    sl_action_logits_t, sl_value = sl_model(state)
  
    # Sample next action from the action probability distribution
    dist = tfd.Categorical(logits=action_logits_t[0])
    action = dist.sample()

    # Store critic values
    values = values.write(t, tf.squeeze(value))

    # Store log probability of the action chosen
    action_logits_ts = action_logits_ts.write(t, action_logits_t[0])
    sl_action_logits_ts = sl_action_logits_ts.write(t, sl_action_logits_t[0])
    
    action_logits = action_logits.write(t, action_logits_t[0, action])
  
    # Apply action to the environment to get next state and reward
    state, reward, done = tf_env_step(action)
    state.set_shape(initial_state_shape)
    
    reward_sum += int(reward)
    reward_sum.set_shape(reward_sum_shape)
  
    # Store reward
    rewards = rewards.write(t, reward)
    if tf.cast(done, tf.bool):
      break

  action_logits = action_logits.stack()
  values = values.stack()
  rewards = rewards.stack()

  action_logits_ts = action_logits_ts.stack()
  sl_action_logits_ts = sl_action_logits_ts.stack()
  
  return action_logits, values, rewards, action_logits_ts, sl_action_logits_ts

### 2. Computing expected returns

The sequence of rewards for each timestep $t$, $\{r_{t}\}^{T}_{t=1}$ collected during one episode is converted into a sequence of expected returns $\{G_{t}\}^{T}_{t=1}$ in which the sum of rewards is taken from the current timestep $t$ to $T$ and each reward is multiplied with an exponentially decaying discount factor $\gamma$:

$$G_{t} = \sum^{T}_{t'=t} \gamma^{t'-t}r_{t'}$$

Since $\gamma\in(0,1)$, rewards further out from the current timestep are given less weight.

Intuitively, expected return simply implies that rewards now are better than rewards later. In a mathematical sense, it is to ensure that the sum of the rewards converges.

To stabilize training, the resulting sequence of returns is also standardized (i.e. to have zero mean and unit standard deviation).


In [8]:
def get_expected_return(
    rewards: tf.Tensor, 
    gamma: float, 
    standardize: bool = True) -> tf.Tensor:
  """Compute expected returns per timestep."""

  n = tf.shape(rewards)[0]
  returns = tf.TensorArray(dtype=tf.float32, size=n)

  # Start from the end of `rewards` and accumulate reward sums
  # into the `returns` array
  rewards = tf.cast(rewards[::-1], dtype=tf.float32)
  discounted_sum = tf.constant(0.0)
  discounted_sum_shape = discounted_sum.shape
  for i in tf.range(n):
    reward = rewards[i]
    discounted_sum = reward + gamma * discounted_sum
    discounted_sum.set_shape(discounted_sum_shape)
    returns = returns.write(i, discounted_sum)
    
  returns = returns.stack()[::-1]

  if standardize:
    returns = ((returns - tf.math.reduce_mean(returns)) / 
               (tf.math.reduce_std(returns) + eps))

  return returns

### 3. The actor-critic loss

Since a hybrid actor-critic model is used, the chosen loss function is a combination of actor and critic losses for training, as shown below:

$$L = L_{actor} + L_{critic}$$

#### Actor loss

The actor loss is based on [policy gradients with the critic as a state dependent baseline](https://www.youtube.com/watch?v=EKqxumCuAAY&t=62m23s) and computed with single-sample (per-episode) estimates.

$$L_{actor} = -\sum^{T}_{t=1} log\pi_{\theta}(a_{t} | s_{t})[G(s_{t}, a_{t})  - V^{\pi}_{\theta}(s_{t})]$$

where:
- $T$: the number of timesteps per episode, which can vary per episode
- $s_{t}$: the state at timestep $t$
- $a_{t}$: chosen action at timestep $t$ given state $s$
- $\pi_{\theta}$: is the policy (actor) parameterized by $\theta$
- $V^{\pi}_{\theta}$: is the value function (critic) also parameterized by $\theta$
- $G = G_{t}$: the expected return for a given state, action pair at timestep $t$

A negative term is added to the sum since the idea is to maximize the probabilities of actions yielding higher rewards by minimizing the combined loss.

<br>

##### Advantage

The $G - V$ term in our $L_{actor}$ formulation is called the [advantage](https://spinningup.openai.com/en/latest/spinningup/rl_intro.html#advantage-functions), which indicates how much better an action is given a particular state over a random action selected according to the policy $\pi$ for that state.

While it's possible to exclude a baseline, this may result in high variance during training. And the nice thing about choosing the critic $V$ as a baseline is that it trained to be as close as possible to $G$, leading to a lower variance.

In addition, without the critic, the algorithm would try to increase probabilities for actions taken on a particular state based on expected return, which may not make much of a difference if the relative probabilities between actions remain the same.

For instance, suppose that two actions for a given state would yield the same expected return. Without the critic, the algorithm would try to raise the probability of these actions based on the objective $J$. With the critic, it may turn out that there's no advantage ($G - V = 0$) and thus no benefit gained in increasing the actions' probabilities and the algorithm would set the gradients to zero.

<br>

#### Critic loss

Training $V$ to be as close possible to $G$ can be set up as a regression problem with the following loss function:

$$L_{critic} = L_{\delta}(G, V^{\pi}_{\theta})$$

where $L_{\delta}$ is the [Huber loss](https://en.wikipedia.org/wiki/Huber_loss), which is less sensitive to outliers in data than squared-error loss.


In [9]:
cce_loss = tf.keras.losses.CategoricalCrossentropy()
cce_loss_logits = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

def compute_supervised_loss(
    action_logits: tf.Tensor,  
    actions: tf.Tensor) -> tf.Tensor:
  """Computes the combined actor-critic loss."""

  actions_onehot = tf.one_hot(actions, num_actions)
  
  action_loss = cce_loss_logits(actions_onehot, action_logits)

  return action_loss

In [10]:
mse_loss = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.SUM)

def compute_rl_loss(
    action_logits: tf.Tensor,  
    values: tf.Tensor,  
    returns: tf.Tensor) -> tf.Tensor:
  """Computes the combined actor-critic loss."""

  advantage = returns - values

  action_probs = tf.nn.softmax(action_logits)
  action_log_probs = tf.math.log(action_probs)
  actor_loss = -tf.math.reduce_sum(action_log_probs * advantage)

  critic_loss = mse_loss(values, returns)

  return actor_loss + critic_loss

In [11]:
kl = tf.keras.losses.KLDivergence()

def compute_kl_loss(
    action_logits_ts: tf.Tensor,  
    sl_action_logits_ts: tf.Tensor) -> tf.Tensor:
  """Computes the combined actor-critic loss."""

  dist = tfd.Categorical(logits=action_logits_ts)
  sl_dist = tfd.Categorical(logits=sl_action_logits_ts) 

  kl_loss = tfd.kl_divergence(dist, sl_dist)
  kl_loss = tf.reduce_mean(kl_loss)

  return kl_loss

### 4. Defining the training step to update parameters

All of the steps above are combined into a training step that is run every episode. All steps leading up to the loss function are executed with the `tf.GradientTape` context to enable automatic differentiation.

This tutorial uses the Adam optimizer to apply the gradients to the model parameters.

The sum of the undiscounted rewards, `episode_reward`, is also computed in this step. This value will be used later on to evaluate if the success criterion is met.

The `tf.function` context is applied to the `train_step` function so that it can be compiled into a callable TensorFlow graph, which can lead to 10x speedup in training.


In [12]:
supervised_optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)


@tf.function
def train_supervised_step(
    model: tf.keras.Model, 
    optimizer: tf.keras.optimizers.Optimizer) -> tf.Tensor:
  """Runs a model training step."""

  with tf.GradientTape() as tape:
    # Run the model for one episode to collect training data
    action_logits, actions = run_supervised_episode(model) 

    # Convert training data to appropriate TF tensor shapes
    action_logits, actions = [tf.expand_dims(x, 1) for x in [action_logits, actions]] 

    # Calculating loss values to update our network
    loss = compute_supervised_loss(action_logits, actions)
    
    regularization_loss = tf.reduce_sum(model.losses)
    loss = loss + regularization_loss

  # Compute the gradients from the loss
  grads = tape.gradient(loss, model.trainable_variables)

  # Apply the gradients to the model's parameters
  supervised_optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
  return loss

In [13]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)

@tf.function
def train_rl_step(
    initial_state: tf.Tensor, 
    initial_reward_sum: tf.Tensor, 
    model: tf.keras.Model, 
    optimizer: tf.keras.optimizers.Optimizer, 
    gamma: float, 
    max_steps_per_episode: int) -> tf.Tensor:
  """Runs a model training step."""

  with tf.GradientTape() as tape:
    # Run the model for one episode to collect training data
    action_logits, values, rewards, action_logits_ts, sl_action_logits_ts = run_reinforcement_episode(
        initial_state, initial_reward_sum, model, max_steps_per_episode) 

    # Calculate expected returns
    returns = get_expected_return(rewards, gamma)

    # Convert training data to appropriate TF tensor shapes
    action_logits, values, returns = [
        tf.expand_dims(x, 1) for x in [action_logits, values, returns]] 

    # Calculating loss values to update our network
    rl_loss = compute_rl_loss(action_logits, values, returns)
    kl_loss = compute_kl_loss(action_logits_ts, sl_action_logits_ts)
    
    loss = rl_loss + kl_loss

  # Compute the gradients from the loss
  grads = tape.gradient(loss, model.trainable_variables)

  # Apply the gradients to the model's parameters
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  episode_reward = tf.math.reduce_sum(rewards)

  return episode_reward

### 5. Run the training loop

Training is executed by running the training step until either the success criterion or maximum number of episodes is reached.  

A running record of episode rewards is kept in a queue. Once 100 trials are reached, the oldest reward is removed at the left (tail) end of the queue and the newest one is added at the head (right). A running sum of the rewards is also maintained for computational efficiency. 

Depending on your runtime, training can finish in less than a minute.

In [14]:
'''
%%time

max_episodes = 500000

with tqdm.trange(max_episodes) as t:
  for i in t:
    loss = train_supervised_step(model, optimizer)
    #with writer.as_default():
    #  # other model code would go here
    #  tf.summary.scalar("loss", loss, step=i)
    #  writer.flush()
    if i % 100 == 0:
      model.save("LunaLander_SL_Model")
      print("loss: ", loss)
      pass # print(f'Episode {i}: average reward: {avg_reward}')
'''

'\n%%time\n\nmax_episodes = 500000\n\nwith tqdm.trange(max_episodes) as t:\n  for i in t:\n    loss = train_supervised_step(model, optimizer)\n    #with writer.as_default():\n    #  # other model code would go here\n    #  tf.summary.scalar("loss", loss, step=i)\n    #  writer.flush()\n    if i % 100 == 0:\n      model.save("LunaLander_SL_Model")\n      print("loss: ", loss)\n      pass # print(f\'Episode {i}: average reward: {avg_reward}\')\n'

In [15]:
%%time

min_episodes_criterion = 100
max_episodes = 1000000
max_steps_per_episode = 1000

# Cartpole-v0 is considered solved if average reward is >= 195 over 100 
# consecutive trials
reward_threshold = 195
running_reward = 0

# Discount factor for future rewards
gamma = 0.99

# Keep last episodes reward
episodes_reward: collections.deque = collections.deque(maxlen=min_episodes_criterion)

with tqdm.trange(max_episodes) as t:
  for i in t:
    initial_state = tf.constant(env.reset(), dtype=tf.float32)
    initial_reward_sum = tf.constant(0, dtype=tf.int32)
    episode_reward = int(train_rl_step(
        initial_state, initial_reward_sum, model, optimizer, gamma, max_steps_per_episode))
    
    episodes_reward.append(episode_reward)
    running_reward = statistics.mean(episodes_reward)
  
    t.set_description(f'Episode {i}')
    t.set_postfix(episode_reward=episode_reward, running_reward=running_reward)

    with writer.as_default():
      for step in range(100):
        # other model code would go here
        tf.summary.scalar("running_reward", running_reward, step=i)
        writer.flush()
  
    # Show average episode reward every 10 episodes
    if i % 100 == 0:
      model.save("LunaLander_RL_Model")
      pass # print(f'Episode {i}: average reward: {avg_reward}')
  
    if running_reward > reward_threshold and i >= min_episodes_criterion:  
        break

print(f'\nSolved at episode {i}: average reward: {running_reward:.2f}!')


  0%|                                               | 0/1000000 [00:00<?, ?it/s]2022-02-23 13:39:28.742195: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-02-23 13:39:28.789267: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 4099825000 Hz
Episode 0:   0%| | 0/1000000 [00:03<?, ?it/s, episode_reward=-267, running_rewar2022-02-23 13:39:29.610313: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: LunaLander_RL_Model/assets


Episode 33:   0%| | 34/1000000 [00:07<59:18:12,  4.68it/s, episode_reward=-50, r


KeyboardInterrupt: 

## Visualization

After training, it would be good to visualize how the model performs in the environment. You can run the cells below to generate a GIF animation of one episode run of the model. Note that additional packages need to be installed for OpenAI Gym to render the environment's images correctly in Colab.

In [ ]:

import collections
import gym
import numpy as np
import statistics
import tensorflow as tf
import tqdm
import glob
import random

from matplotlib import pyplot as plt
from tensorflow.keras import layers
from typing import Any, List, Sequence, Tuple

import gym
import os
import tensorflow_probability as tfp

tfd = tfp.distributions

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Create the environment
env = gym.make("LunarLander-v2")

# Set seed for experiment reproducibility
seed = 42
env.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

#model = tf.keras.models.load_model('LunaLander_RL_Model')

class ActorCritic(tf.keras.Model):
  """Combined actor-critic network."""

  def __init__(
      self, 
      num_actions: int, 
      num_hidden_units: int):
    """Initialize."""
    super().__init__()

    self.common_1 = layers.Dense(num_hidden_units*2, activation="relu")
    self.common_2 = layers.Dense(num_hidden_units, activation="relu")
    self.actor = layers.Dense(num_actions)
    self.critic = layers.Dense(1)

  def call(self, inputs: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
    x = self.common_1(inputs)
    x = self.common_2(x)
    
    return self.actor(x), self.critic(x)


num_actions = env.action_space.n  # 2
num_hidden_units = 256

sl_model = ActorCritic(num_actions, num_hidden_units)
sl_model.load_weights("/home/kimbring2/Supervised_A2C/LunaLander_SL_Model/variables/variables")

total_step = 0
reward_sum = 0

episode_list = []
action_list = []
reward_list = []
done_list = []
step_list = []
state_list = []
next_state_list = []
for i_episode in range(452, 10000):
    observation = tf.constant(env.reset(), dtype=tf.float32)
    
    step = 0
    while True:
        step += 1
        env.render()

        observation = tf.expand_dims(observation, 0)
        #observation = tf.expand_dims(observation, 0)
        action_probs, _ = sl_model(observation)
        
        dist = tfd.Categorical(logits=action_probs[0])
        action = int(dist.sample())
        
        #action = np.argmax(np.squeeze(action_probs))
        #print("action: ", action)

        observation_1, reward, done, info = env.step(action)
        observation_1 = tf.constant(observation_1, dtype=tf.float32)
        #print("observation_1.shape: ", observation_1.shape)
        
        reward_sum += reward

        #episode_list.append(i_episode)
        #action_list.append(action)
        #reward_list.append(reward)
        #done_list.append(done)
        #step_list.append(step)
        #state_list.append(observation)
        next_state_list.append(observation_1)

        observation = observation_1
        if done:
            print("Total reward: {:.2f},  Total step: {:.2f}".format(reward_sum, total_step))
            step = 0
            reward_sum = 0  
            observation = env.reset()
            '''
            print("Save data")
            save_data = {'episode': i_episode, 'step': step_list, 
                         'state': state_list, 'next_state': next_state_list, 
                         'action': action_list, 'reward': reward_list, 'done': done_list}

            save_path = '/media/kimbring2/Steam/Supervised_A2C/expert_data'
            save_file = '/data_' + str(i_episode)
            path_npy = save_path + save_file + '.npy'
            np.save(path_npy, save_data)

            episode_list = []
            action_list = []
            reward_list = []
            done_list = []
            step_list = []
            state_list = []
            next_state_list = []
            '''
            break

env.close()


## Next steps

This tutorial demonstrated how to implement the actor-critic method using Tensorflow.

As a next step, you could try training a model on a different environment in OpenAI Gym. 

For additional information regarding actor-critic methods and the Cartpole-v0 problem, you may refer to the following resources:

- [Actor Critic Method](https://hal.inria.fr/hal-00840470/document)
- [Actor Critic Lecture (CAL)](https://www.youtube.com/watch?v=EKqxumCuAAY&list=PLkFD6_40KJIwhWJpGazJ9VSj9CFMkb79A&index=7&t=0s)
- [Cartpole learning control problem \[Barto, et al. 1983\]](http://www.derongliu.org/adp/adp-cdrom/Barto1983.pdf) 

For more reinforcement learning examples in TensorFlow, you can check the following resources:
- [Reinforcement learning code examples (keras.io)](https://keras.io/examples/rl/)
- [TF-Agents reinforcement learning library](https://www.tensorflow.org/agents)
